In [14]:
import os
import json

from itertools import groupby
from config import iam_policy_data
import preprocessiong_iam_policy

def group_permissions_by_service(policy_lists):
    grouped_permissions = {}
    for policy_list in policy_lists:
        if policy_list["Service Effect"] != iam_policy_data.service_effect[0]:
            service = policy_list["Service"]
            if service not in grouped_permissions:
                grouped_permissions[service] = {"Action": [], "Resource": "*"}
            grouped_permissions[service]["Action"].append(policy_list["Actions"])
        else:
            pass
    return grouped_permissions

def generate_iam_policy(grouped_permissions):
    iam_policy = {
        "Version": "2012-10-17",
        "Statement": [],
    }

    for service, actions_resources in grouped_permissions.items():
        statement = {
            "Sid": f"{service.lower()}allowfordevops",
            "Effect": "Allow",
            "Action": actions_resources["Action"],
            "Resource": actions_resources["Resource"],
        }
        iam_policy["Statement"].append(statement)

    return iam_policy

def save_iam_policy_to_file(iam_policy, category, output_directory):
    output_file_path = os.path.join(output_directory, f"{category.lower()}_iam_policy_devops")
    with open(output_file_path, 'w') as json_file:
        json.dump(iam_policy, json_file, indent=4)

# def main():
#     policy_lists = preprocessiong_iam_policy.excel_to_json(iam_policy_data.input_file_path, iam_policy_data.sheet_name)
#     grouped_by_category = {key: list(group) for key, group in groupby(policy_lists, key=lambda x: x['Category'])}
    
#     for category, items in grouped_by_category.items():
#         print(f"Category: {category}")
#         grouped_permissions = group_permissions_by_service(items)
#         iam_policy = generate_iam_policy(grouped_permissions)    
#         save_iam_policy_to_file(iam_policy, category , iam_policy_data.output_directory)

# if __name__ == "__main__":
#     main()


In [15]:
policy_lists = preprocessiong_iam_policy.excel_to_json(iam_policy_data.input_file_path, iam_policy_data.sheet_name)

In [16]:
# Category와 Wildcard로 데이터 정렬
sorted_data = sorted(policy_lists, key=lambda x: (x['Category'], x['Wildcard']))

# Category와 Wildcard로 그룹화
grouped_by_category_and_wildcard = {key: list(group) for key, group in groupby(sorted_data, key=lambda x: (x['Category'], x['Wildcard']))}

# Grouped 결과를 저장할 리스트
grouped_actions = []
personal_actions = []

# 그룹화된 데이터에서 Service Effect가 모두 같은 경우 Action을 하나로 처리
for key, group in grouped_by_category_and_wildcard.items():
    category, wildcard = key
    service_effects = set(item['Service Effect'] for item in group)
    if iam_policy_data.service_effect[0] not in service_effects:
        action = f"{group[0]['Actions'].split(':')[0].lower()}:{wildcard}"
        grouped_actions.append({'Wildcard' : wildcard, 'Action': action, 'Category': category, 'Service': group[0]['Service'], 'Service Effect': service_effects.pop()})

# # Action 값의 ":" 뒤의 값이 policy_lists의 Wildcard 값과 다른 경우 추가
# for item in policy_lists:
#     action_name = item['Actions'].split(':')[1]
#     if any(action_name not in grouped_item['Action'] for grouped_item in grouped_actions):
#         grouped_actions.append(item)


In [17]:
grouped_by_category_and_wildcard

{('Infra',
  'Accept*'): [{'Wildcard': 'Accept*',
   'Actions': 'ec2:AcceptAddressTransfer',
   'Category': 'Infra',
   'Service': 'EC2',
   'Service Effect': '상 (높음)'}, {'Wildcard': 'Accept*',
   'Actions': 'ec2:AcceptReservedInstancesExchangeQuote',
   'Category': 'Infra',
   'Service': 'EC2',
   'Service Effect': '중 (중간)'}, {'Wildcard': 'Accept*',
   'Actions': 'ec2:AcceptVpcPeeringConnection',
   'Category': 'Infra',
   'Service': 'VPC',
   'Service Effect': '중 (중간)'}],
 ('Infra',
  'Allocate*'): [{'Wildcard': 'Allocate*',
   'Actions': 'ec2:AllocateAddress',
   'Category': 'Infra',
   'Service': 'EC2',
   'Service Effect': '중 (중간)'}, {'Wildcard': 'Allocate*',
   'Actions': 'ec2:AllocateHosts',
   'Category': 'Infra',
   'Service': 'EC2',
   'Service Effect': '중 (중간)'}],
 ('Infra',
  'Assign*'): [{'Wildcard': 'Assign*',
   'Actions': 'ec2:AssignIpv6Addresses',
   'Category': 'Infra',
   'Service': 'VPC',
   'Service Effect': '중 (중간)'}, {'Wildcard': 'Assign*',
   'Actions': 'ec2:Ass

In [18]:
#### grouped_actions_dict를 'Wildcard'를 키로 하는 딕셔너리로 변경
grouped_actions_dict_by_wildcard = {item['Wildcard']: item for item in grouped_actions}

# 삭제할 인덱스를 저장할 리스트 초기화
indices_to_remove = []

# policy_lists 순회
for idx, policy in enumerate(policy_lists):
    wildcard = policy.get('Wildcard')
    category = policy.get('Category')
    service = policy.get('Service')
    print(category)

    # wildcard이 grouped_actions_dict에 존재하면서 값이 일치하는 경우
    try:
        if grouped_actions_dict_by_wildcard[wildcard]['Wildcard'] == wildcard and grouped_actions_dict_by_wildcard[wildcard]['Category'] == category and grouped_actions_dict_by_wildcard[wildcard]['Service'] == service:
            indices_to_remove.append(idx)
            print(idx)
    except KeyError:
        # KeyError가 발생하면 예외 처리
        print(f"KeyError: {wildcard} not found in grouped_actions_dict_by_wildcard")

# 저장된 인덱스를 역순으로 정렬하여 뒤에서부터 삭제
for idx in reversed(indices_to_remove):
    del policy_lists[idx]


Infra
KeyError: Accept* not found in grouped_actions_dict_by_wildcard
Infra
KeyError: Accept* not found in grouped_actions_dict_by_wildcard
Infra
KeyError: Accept* not found in grouped_actions_dict_by_wildcard
Network
KeyError: Activate* not found in grouped_actions_dict_by_wildcard
Security
KeyError: Add* not found in grouped_actions_dict_by_wildcard
Security
KeyError: Add* not found in grouped_actions_dict_by_wildcard
Security
KeyError: Add* not found in grouped_actions_dict_by_wildcard
Storage
KeyError: Add* not found in grouped_actions_dict_by_wildcard
Storage
KeyError: Add* not found in grouped_actions_dict_by_wildcard
Storage
KeyError: Add* not found in grouped_actions_dict_by_wildcard
Storage
KeyError: Add* not found in grouped_actions_dict_by_wildcard
Security
KeyError: Add* not found in grouped_actions_dict_by_wildcard
Security
KeyError: Add* not found in grouped_actions_dict_by_wildcard
Storage
KeyError: Add* not found in grouped_actions_dict_by_wildcard
Storage
KeyError: Add

In [19]:
grouped_actions_dict_by_wildcard = {item['Wildcard']: item for item in grouped_actions}

In [20]:
# grouped_actions_dict_by_wildcard
output_file_path = os.path.join('./', "group_test.json")
with open(output_file_path, 'w') as json_file:
    json.dump(grouped_actions_dict_by_wildcard, json_file, indent=4)

In [21]:
policy_lists[200:250]

[{'Wildcard': 'Delete*',
  'Actions': 'ec2:DeleteRoute',
  'Category': 'Infra',
  'Service': 'VPC',
  'Service Effect': '상 (높음)'},
 {'Wildcard': 'Delete*',
  'Actions': 'ec2:DeleteRouteTable',
  'Category': 'Infra',
  'Service': 'VPC',
  'Service Effect': '상 (높음)'},
 {'Wildcard': 'Delete*',
  'Actions': 'ec2:DeleteSubnet',
  'Category': 'Infra',
  'Service': 'VPC',
  'Service Effect': '상 (높음)'},
 {'Wildcard': 'Delete*',
  'Actions': 'ec2:DeleteSubnetCidrReservation',
  'Category': 'Infra',
  'Service': 'VPC',
  'Service Effect': '상 (높음)'},
 {'Wildcard': 'Delete*',
  'Actions': 'ec2:DeleteTrafficMirrorFilter',
  'Category': 'Infra',
  'Service': 'VPC',
  'Service Effect': '상 (높음)'},
 {'Wildcard': 'Delete*',
  'Actions': 'ec2:DeleteTrafficMirrorFilterRule',
  'Category': 'Infra',
  'Service': 'VPC',
  'Service Effect': '상 (높음)'},
 {'Wildcard': 'Delete*',
  'Actions': 'ec2:DeleteTrafficMirrorSession',
  'Category': 'Infra',
  'Service': 'VPC',
  'Service Effect': '상 (높음)'},
 {'Wildcard': 

In [22]:
grouped_actions

[{'Wildcard': 'Allocate*',
  'Action': 'ec2:Allocate*',
  'Category': 'Infra',
  'Service': 'EC2',
  'Service Effect': '중 (중간)'},
 {'Wildcard': 'Assign*',
  'Action': 'ec2:Assign*',
  'Category': 'Infra',
  'Service': 'VPC',
  'Service Effect': '중 (중간)'},
 {'Wildcard': 'Attach*',
  'Action': 'ec2:Attach*',
  'Category': 'Infra',
  'Service': 'VPC',
  'Service Effect': '중 (중간)'},
 {'Wildcard': 'Confirm*',
  'Action': 'ec2:Confirm*',
  'Category': 'Infra',
  'Service': 'EC2',
  'Service Effect': '하 (낮음)'},
 {'Wildcard': 'Copy*',
  'Action': 'ec2:Copy*',
  'Category': 'Infra',
  'Service': 'EC2',
  'Service Effect': '중 (중간)'},
 {'Wildcard': 'Describe*',
  'Action': 'ec2:Describe*',
  'Category': 'Infra',
  'Service': 'EC2',
  'Service Effect': '하 (낮음)'},
 {'Wildcard': 'Disable*',
  'Action': 'ec2:Disable*',
  'Category': 'Infra',
  'Service': 'EC2',
  'Service Effect': '중 (중간)'},
 {'Wildcard': 'Discover*',
  'Action': 'ecs:Discover*',
  'Category': 'Infra',
  'Service': 'ECS',
  'Service 

In [23]:
print(len(policy_lists))
print(len(grouped_actions))

953
39


In [24]:
441 + 67

508

In [25]:
policy_lists.extend(grouped_actions)

In [26]:
len(policy_lists)

992

In [27]:
group_dict = preprocessiong_iam_policy.group_dict_by_key(policy_lists, 'Category')

In [28]:
for category, items in group_dict.items():
    print(f"{category} IAM Policy created for devops")
    grouped_permissions = group_permissions_by_service(items)
    iam_policy = generate_iam_policy(grouped_permissions)    
    save_iam_policy_to_file(iam_policy, category, iam_policy_data.output_directory)


Infra IAM Policy created for devops


KeyError: 'Actions'